In [24]:
import pySWAP
import importlib
from pandas import DataFrame

importlib.reload(pySWAP)
from pySWAP import project_dir
from pySWAP import swapmodel as swp
from pySWAP import metadata as met
from pySWAP import meteo
from pySWAP import drainage as dra
from pySWAP import crop

Hello from pySWAP/__init__.py


#### 0. Metadata

In [25]:
metadata = met.Metadata(author='John Smith',
                       institution='University of Somewhere',
                       email='john@somewhere.com',
                       project_name='TESTMODEL3',
                       swap_ver='4.2')

ml1 = swp.SwapModel(metadata=metadata)  # create a model instance with given header

#### 1. Environment

In [26]:
ml1.set_environment()

#### 2. Simulation period

In [27]:
start = '2002-01-01'
end = '2004-12-31'

ml1.set_simulation_period(start=start,
                          end=end)

#### 3. Output dates

In [28]:
# Optional settings from the #1 example:
# outdatin = ['2022-01-01', '2022-06-01']
# outdat = ['2022-01-01', '2022-06-01']
datefix = '21 11'

ml1.set_output_dates(nprintday=1,
                     swmonth=True,
                     swyrvar=False,
                     datefix=datefix)

#### 4. Output files

In [29]:
# optional CSV settings from #1 example:
# inlist_csv_tz = ['wc', 'h', 'conc']
# inlist_csv = ['rain', 'irrig', 'interc', 'runoff', 'drainage', 'dstor', 'epot', 'eact', 'tpot', 'tact', 'qbottom', 'gwl']
# dzwnew = [10.0, 10.0, 10.0, 20.0, 30.0, 50.0]
# critdevmasbal: float = 0.00001
# numnodnew: int = 6
ml1.set_output_files()

In [30]:
ml1.print_section_params('general')

PROJECT              = TESTMODEL3
FILENAME             = TESTMODEL3.swp
SWAPVER              = 4.2
TIMESTAMP            = 2023-06-08 14:14:48
AUTHOR               = John Smith
INSTITUTION          = University of Somewhere
EMAIL                = john@somewhere.com
COMMENT              = No comment
PATHWORK             = .\
PATHATM              = .\
PATHCROP             = .\
PATHDRAIN            = .\
SWSCRE               = 0
SWERROR              = False
START                = 2002-01-01
END                  = 2004-12-31
NPRINTDAY            = 1
SWMONTH              = True
SWYRVAR              = False
DATEFIX              = 21 11
OUTFIL               = result
SWHEADER             = False
SWWBA                = False
SWEND                = False
SWVAP                = True
SWBAL                = False
SWBLC                = True
SWSBA                = True
SWATE                = False
SWBMA                = False
SWDRF                = False
SWSWB                = False
SWINI             

#### 5. Crop

In [31]:
rds = 200.0
# There is a brute force solution here to encluse the cropname and cropfile in single quotes. It is not elegant, but it works.
croprotation = DataFrame({'INITCROP': [1, 1, 1],
                          'CROPSTART': ['2002-05-01', '2003-05-10', '2004-01-01'],  # in the originl file it was ['01-may-02']
                          'CROPEND': ['2002-10-15', '2003-09-29', '2004-12-31'],
                          'CROPNAME': ["'maize'", "'potato'", "'grass'"],
                          'CROPFIL': ["'maizes'", "'potatod'", "'grassd'"],
                          'CROPTYPE': [1, 2, 3],})



# prepare the crop files
maize_path = r"C:\Users\zawad\PycharmProjects\pySWAP_django\djangoSWAP\setup_model\pySWAP\pySWAP\data\.crp\maizes.crp"
potato_path = r"C:\Users\zawad\PycharmProjects\pySWAP_django\djangoSWAP\setup_model\pySWAP\pySWAP\data\.crp\potatod.crp"
grass_path = r"C:\Users\zawad\PycharmProjects\pySWAP_django\djangoSWAP\setup_model\pySWAP\pySWAP\data\.crp\grassd.crp"
file_names = ['maizes', 'potatod', 'grassd']

crop = crop.Crop(file_paths=[maize_path, potato_path, grass_path],
                 file_names=file_names)

ml1.set_crop(swcrop=True,
             croprotation=croprotation,
             rds=rds,
             crop=crop)

# after defining the crop rotation, SWAP will look for the crop files under the specified PATHCROP

### 6. Irrigation

In [32]:
## optional parameters:
# irgfil: str = 'testirri'
# startirr: str = '30 3'
# endirr: str = '31 12'
# cirrs: float = 0.0
# isuas: int = 1
# phFieldCapacity: float = -100.0
# tcs: int = 1
# phormc: int = 0
# cirrthres: float = 8.0
# perirrsurp: float = 10.0
# irgthreshold: float = 1.0
# tcsfix: int = 0
# dcrit: float = -30.0
# irgdayfix: int = 7

irigevents = DataFrame({'IRDATE': ['2002-01-05'],
                        'IRDEPTH': [5.0],
                        'IRCONC': [1000.0],
                        'IRTYPE': [1]})

ml1.set_irrigation(swirfix=True,
                   schedule=False,
                   swirgfil=False,
                   irigevents=irigevents)


#### 7. Meteorological data

In [33]:
## Optional parameters:
# rainflux = DataFrame({'TIME': [1.0, 360.0],
#                      'RAINFLUX': [20.0, 20.0]})

csv_path = r"C:\Users\zawad\PycharmProjects\pySWAP_django\djangoSWAP\setup_model\pySWAP\pySWAP\data\daily_basic_weather.csv"
met1 = meteo.Meteo(metadata)  # create a meteo instance with given header

met1.load_from_csv(csv_path=csv_path,
                   station='238',
                   station_lon=52.0,
                   station_lat=52.0,
                   station_alt=60.5)

ml1.set_meteo(swetr=False,
              meteo_data=met1,
              altw=10.0,
              angstroma=0.25,
              angstromb=0.5,
              swdivide=True,
              swmetdetail=False)  # set meteo data and store the csv file in the main model instance


#### 8. initial soil moisture (soil-water section)

In [34]:
# Optional params
# head_soildepth = DataFrame({'ZI': [-0.5, -195.0],
#                             'H': [-93.0, 120.0]})
# inifil = 'testinifil'

ml1.set_init_soil_moisture(swinco=2,
                           gwli=75.1)


#### 9. Surface flow

In [35]:
## Optional paremeters:
# swpondmx = 0
# pondmx = 0.2
# rsro = 0.5
# rsroexp = 1.0
# swrunon = 0
# rufil = 'swap.inc'
# mxpondtb = DataFrame({'DATEPMX': ['2002-01-01', '2004-12-31'],
#                       'PONDMXTB': [0.2, 0.2]})

ml1.set_surface_flow(swpondmx=0,
                     swrunon=False,
                     rsro=0.5,
                     rsroexp=1.0,
                     pondmx=0.2,)


#### 10. Soil evaporation (soil-water section)

In [36]:
# # Optional parameters:
# cfbs = 0.5
# rsoil = 30.0
# cofredbl = 0.35
# rsigni = 0.5
# cofredbo = 0.35

ml1.set_soil_evap(cfevappond=1.25,
                  swcfbs=False,
                  swredu=1,
                  rsoil=30.0,
                  rsigni=0.5,
                  cofredbl=0.35,)

#### 11. Soil profile (soil-water section)

In [37]:
soilprofile = DataFrame({'ISUBLAY': [1, 2, 3, 4],
                         'ISOILLAY': [1, 1, 2, 2],
                         'HSUBLAY': [10.0, 20.0, 30.0, 140.0],
                         'HCOMP': [1.0, 5.0, 5.0, 10.0],
                         'NCOMP': [10, 4, 6, 14]})

ml1.set_soil_profile(soilprofile=soilprofile)


#### 12. Soil parameters (soil-water section)

In [38]:
soilhydrfunc = DataFrame({'ORES': [0.01, 0.02],
                          'OSAT': [0.42, 0.38],
                          'ALFA': [0.0276, 0.0213],
                          'NPAR': [1.491, 1.951],
                          'KSATFIT': [12.52, 12.68],
                          'LEXP': [-1.060, 0.168],
                          'ALFAW': [0.0542, 0.0426],
                          'H_ENPR': [0.0, 0.0],
                          'KSATEXM': [12.52, 12.68],
                          'BDENS': [1315.0, 1315.0]})

# # Optional parameters:
# filenamesophy = 'topsoil_sand_b2.csv' 'subsoil_sand_o2.csv'
# tau = 0.2

ml1.set_soil_parameters(swsophy=False,
                        swhyst=0,
                        swmacro=False,
                        soilhydrfunc=soilhydrfunc,)


#### 13. Snow and frost (soil-water section)

In [39]:
# Optional parameters:
# snowinco = 22.0
# teprrain = 2.0
# teprsnow = -2.0
# snowcoef = 0.3
# tfroststa = 0.0
# tfrostend = -1.0

ml1.set_snow_frost(swsnow=True,
                   swfrost=False,)


In [40]:
ml1.print_section_params('soil_water')

SWINCO               = 2
GWLI                 = 75.1
SWPONDMX             = 0
SWRUNON              = False
PONDMX               = 0.2
RSRO                 = 0.5
RSROEXP              = 1.0
SWCFBS               = False
SWREDU               = 1
CFEVAPPOND           = 1.25
RSOIL                = 30.0
COFREDBL             = 0.35
RSIGNI               = 0.5
SWSOPHY              = False
SWHYST               = 0
SWMACRO              = False
SWSNOW               = True
SWFROST              = False
   ISUBLAY  ISOILLAY  HSUBLAY  HCOMP  NCOMP
0        1         1     10.0    1.0     10
1        2         1     20.0    5.0      4
2        3         2     30.0    5.0      6
3        4         2    140.0   10.0     14
   ORES  OSAT    ALFA   NPAR  KSATFIT   LEXP   ALFAW  H_ENPR  KSATEXM   BDENS
0  0.01  0.42  0.0276  1.491    12.52 -1.060  0.0542     0.0    12.52  1315.0
1  0.02  0.38  0.0213  1.951    12.68  0.168  0.0426     0.0    12.68  1315.0


#### 14. Richards parameters (soil-water section)

In [41]:
# Optional parameters:
ml1.set_richards_params(dtmin=0.000001,
                        dtmax=0.04,
                        gwlconv=100.0,
                        critdevh1cp=0.01,
                        critdevh2cp=0.1,
                        critdevponddt=0.0001,
                        maxit=30,
                        maxbacktr=3,
                        swkmean=1,
                        swkimpl=False,)

#### 15. Drainage

In [42]:
# drainage data are now a separate object. For now, it is the entire default .dra file.
dra_path = r'../data/'
dra = dra.Drainage('swap',
                    dra_path,)

ml1.set_drainage(swdra=True,
                 drainage=dra,)


#### 16. Bottom boundary

In [43]:
# Optional parameters:
# This section contains many conditional parameters and tables. For details, refer to the docs.

ml1.set_bottom_boundary(swbbcfile=False,
                        swbotb=6)

#### 17. Heat flow

In [44]:
# # Optional parameters:
# swhea = 1
# swcalt = 2
# tampli = 10
# tmean = 15
# timref = 90
# ddamp = 50
# swtopbhea = 1
# tsoilfile = 'swap'
# swbotbhea = 1
## Optional tables:
# soiltextures = DataFrame({'PSAND': [0.8, 0.8],
#                        'PSILT': [0.15, 0.15],
#                        'PCLAY': [0.05, 0.05],
#                        'ORGMAT': [0.1, 0.1]})
# initsoil = DataFrame({'ZH': [-10.0, -40.0, -70.0, -95.0],
#                        'TSOIL': [15.0, 12.0, 10.0, 9.0]})
# bbctsoil = DataFrame({'DATET': ['2002-01-01', '2002-06-30', '2002-12-23'],
#                        'TBOT': [-15.0, -20.0, -10.0]})

ml1.set_heatflow(swhea=False)

In [45]:
## Optional parameters:
# There are many optional parameters and tables in this section. For details, refer to the docs.

ml1.set_solute_settings(swsolu=False)

In [46]:
await ml1.model_to_db()


Model 'TESTMODEL3' was successfully added to the database.
